In [2]:
api_key = ""

In [3]:
from langchain.llms import OpenAI


llm = OpenAI(
    openai_api_key = api_key
)

In [95]:
result = llm("Write a very very short poem")
print(result)



Life's a bliss
So why not kiss?
A sweet embrace
A timeless trace.


In [97]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(
    openai_api_key = api_key
)

code_promt = PromptTemplate(
    template  = "Write a very short {language} function that will {task}",
    input_variables = ["language", "task"]
)

code_chain = LLMChain(
    llm = llm,
    prompt = code_promt
)

result = code_chain({
    "language": "python",
    "task": "return a list of numbers"
})

print(result)


{'language': 'python', 'task': 'return a list of numbers', 'text': '\n\ndef get_nums():\n  nums = [1, 2, 3, 4, 5]\n  return nums'}


In [99]:
from pprint import pprint

print(type(result))
pprint(result)

<class 'dict'>
{'language': 'python',
 'task': 'return a list of numbers',
 'text': '\n\ndef get_nums():\n  nums = [1, 2, 3, 4, 5]\n  return nums'}


In [149]:
# Connecting Chains Together
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from dotenv import load_dotenv
import argparse

load_dotenv()

parser = argparse.ArgumentParser()
parser.add_argument("--task", default="returns a list of numbers")
parser.add_argument("--language", default="python")
args = parser.parse_args()

llm = OpenAI()

code_promt = PromptTemplate(
    input_variables=["task","language"],
    template="Write a very short {language} function that will {task}"
)

test_prompt = PromptTemplate(
    input_variables=["language", "code"],
    template="Write a test for the following {language} code:\n{code}"

)
code_chain = LLMChain(
    llm = llm,
    prompt = code_promt,
    output_key = "code"
)

test_chain = LLMChain(
    llm = llm,
    prompt = test_prompt,
    output_key = "test"
)

chain = SequentialChain(
    chains = [code_chain, test_chain],
    input_variables = ["task", "language"],
    output_variables = ["test", "code"]
)

result = chain({
    "language": args.language,
    "task": args.task
})

print(">>>>>>> GENERATED CODE <<<<<<<")
print(result["code"])

print(">>>>>>> GENERATED TEST <<<<<<<")
print(result["test"])

usage: ipykernel_launcher.py [-h] [--task TASK] [--language LANGUAGE]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\Atind\AppData\Roaming\jupyter\runtime\kernel-v2-10152gk3QbhK4Poj6.json


SystemExit: 2

C:\Users\Atind\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [144]:
import PyPDF2

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Replace 'Resume.pdf' with the path to your resume file
resume_text = extract_text_from_pdf('Resume.pdf')
# pprint(resume_text)


In [145]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(
    openai_api_key = api_key
)

code_promt = PromptTemplate(
    template  = "Identify the {details} in this resume:\n\n{resume_text}",
    input_variables = ["details", "resume_text"]
)

code_chain = LLMChain(
    llm = llm,
    prompt = code_promt
)

# result = code_chain({
#     "details": "Name of the candidate",
#     "resume_text": resume_text
# })

extracted_details = {}
for detail in ["Name of the candidate", "soft skills", "programming skills", "programming languages"]:
    result = code_chain({
        "details": detail,
        "resume_text": resume_text
    })
    extracted_details[detail] = result['text']


pprint(extracted_details)


{'Name of the candidate': '\n \nAnswer: Robert Stanhill',
 'programming languages': '\n'
                          ' \n'
                          '\n'
                          'Programming Languages: C/C++, Python (Numpy & '
                          'Pandas), TypeScript',
 'programming skills': '\n'
                       '• Runner  Up  – 2021  BNM  Institute  of Technology '
                       'IEEE  Hackathon  \n'
                       ' \n'
                       'Programming skills: C/C++, Python (Numpy & Pandas), '
                       'TypeScript',
 'soft skills': '\n'
                '\n'
                'Soft Skills: Problem-solving, Collaborative Leadership, Good '
                'listener and Effective communicator, Eager & Enthusiastic '
                'learner.'}


In [125]:
print(result['text'])




Programming skills: C/C++, Python (Numpy & Pandas), TypeScript, AngularJS, Angular13


In [146]:
def create_cypher_queries(extracted_details):
    queries = []

    # Create the Person node
    person_name = extracted_details['Name of the candidate'].split(': ')[-1].strip()
    queries.append(f"CREATE (person:Person {{name: '{person_name}'}})")

    # Create nodes for programming skills and relationships to the Person
    programming_skills = extracted_details['programming skills'].split(': ')[-1].split(', ')
    for skill in programming_skills:
        skill_id = skill.replace(" ", "_").replace("&", "And")  # Creating a simple ID
        queries.append(f"MERGE (skill:Skill:Programming {{name: '{skill}', id: '{skill_id}'}})")
        queries.append(f"MATCH (person:Person {{name: '{person_name}'}}), (skill:Skill {{id: '{skill_id}'}}) CREATE (person)-[:HAS_SKILL]->(skill)")

    # Create nodes for soft skills and relationships to the Person
    soft_skills = extracted_details['soft skills'].split(': ')[-1].split(', ')
    for skill in soft_skills:
        skill_id = skill.replace(" ", "_").replace("&", "And")  # Creating a simple ID
        queries.append(f"MERGE (skill:Skill:Soft {{name: '{skill}', id: '{skill_id}'}})")
        queries.append(f"MATCH (person:Person {{name: '{person_name}'}}), (skill:Skill {{id: '{skill_id}'}}) CREATE (person)-[:HAS_SKILL]->(skill)")

    return queries

# # Example usage
# extracted_details = {
#     'Name of the candidate': 'Answer: Robert Stanhill',
#     'programming skills': 'Programming Skills: C/C++, Python (Numpy & Pandas), TypeScript, AngularJS, Angular13',
#     'soft skills': 'Soft Skills: Problem-solving, Collaborative Leadership, Good listener and Effective communicator, Eager & Enthusiastic learner.'
# }

cypher_queries = create_cypher_queries(extracted_details)
for query in cypher_queries:
    print(query)


CREATE (person:Person {name: 'Robert Stanhill'})
MERGE (skill:Skill:Programming {name: 'C/C++', id: 'C/C++'})
MATCH (person:Person {name: 'Robert Stanhill'}), (skill:Skill {id: 'C/C++'}) CREATE (person)-[:HAS_SKILL]->(skill)
MERGE (skill:Skill:Programming {name: 'Python (Numpy & Pandas)', id: 'Python_(Numpy_And_Pandas)'})
MATCH (person:Person {name: 'Robert Stanhill'}), (skill:Skill {id: 'Python_(Numpy_And_Pandas)'}) CREATE (person)-[:HAS_SKILL]->(skill)
MERGE (skill:Skill:Programming {name: 'TypeScript', id: 'TypeScript'})
MATCH (person:Person {name: 'Robert Stanhill'}), (skill:Skill {id: 'TypeScript'}) CREATE (person)-[:HAS_SKILL]->(skill)
MERGE (skill:Skill:Soft {name: 'Problem-solving', id: 'Problem-solving'})
MATCH (person:Person {name: 'Robert Stanhill'}), (skill:Skill {id: 'Problem-solving'}) CREATE (person)-[:HAS_SKILL]->(skill)
MERGE (skill:Skill:Soft {name: 'Collaborative Leadership', id: 'Collaborative_Leadership'})
MATCH (person:Person {name: 'Robert Stanhill'}), (skill:Ski

In [120]:

def create_cypher_queries(result):
    queries = []

    # Extracting entity name from the 'text' field in the result
    entity_name = result['text'].split(": ")[-1].strip()

    # Assuming the entity is a 'Person', since the example given is a candidate's name
    queries.append(f"CREATE (:Person {{name: '{entity_name}'}})")

    # Add additional logic here for other types of entities if needed

    # If there were relationships, they would be processed here
    # For now, it seems we only have a single entity

    return queries

cypher_queries = create_cypher_queries(result)
print(cypher_queries)


["CREATE (:Person {name: 'Robert Stanhill'})"]


In [147]:
from neo4j import GraphDatabase

url = "neo4j+s://04b59de7.databases.neo4j.io"  # Replace with your Neo4j instance URI
username = "neo4j"  # Replace with your username
password = "9aCyMIcwMivgow5OvvkHbJ9svjUkzviHu5du0c5FZbw"  # Replace with your password

driver = GraphDatabase.driver(url, auth=(username, password))

def add_data_to_neo4j(queries):
    with driver.session() as session:
        for query in queries:
            session.run(query)

add_data_to_neo4j(cypher_queries)


In [101]:
resume_prompt_template = """
From the resume below, extract the details of Person name, Experience and Education and the following Entities & relationships and represent them in a key-value pair format which is perfectly a JSON format and also remove anything that appears outside the curly braces { }:
0. ALWAYS FINISH THE OUTPUT. Never send partial responses.
1. Identify and list these Entity types in a key-value pair format:
    Entity Types:
    - 'Person": {'id': string, 'name': string}; // The candidate. `id` is a unique identifier, in camel-case, based on the person's name.
    - 'Experience': {'id': string, 'role': string, 'organization': string}; // Professional experiences. `id` is a unique identifier for each experience.
    - 'Education': {'id': string, 'institution': string, 'degree': string, 'year': string}; // Educational background. `id` is a unique identifier for each education record.

2. Define relationships in a key-value pair format:
    Relationship Types:
    - {'personid': string, 'relationship': 'HAS_EXPERIENCE', 'experienceid': string}
    - {'personid': string, 'relationship': 'HAS_EDUCATION', 'educationid': string}

3. The output should be structured as:
{
    "entities": [{'label': 'Person', 'properties': {...}}, ...],
    "relationships": [{'personid': string, 'relationship': string, 'relatedEntityId': string}, ...]
}

Resume Content:
$ctext

"""

# Combine the prompt with the resume text
resume_input_text = f"{resume_prompt_template}\n\n{resume_text}"
print(resume_input_text)
# # Get model response
# response = text2text_pipeline(input_text)
# extracted_info = response[0]['generated_text']

# # Print the extracted information
# print(extracted_info)


From the resume below, extract the details of Person name, Experience and Education and the following Entities & relationships and represent them in a key-value pair format which is perfectly a JSON format and also remove anything that appears outside the curly braces { }:
0. ALWAYS FINISH THE OUTPUT. Never send partial responses.
1. Identify and list these Entity types in a key-value pair format:
    Entity Types:
    - 'Person": {'id': string, 'name': string}; // The candidate. `id` is a unique identifier, in camel-case, based on the person's name.
    - 'Experience': {'id': string, 'role': string, 'organization': string}; // Professional experiences. `id` is a unique identifier for each experience.
    - 'Education': {'id': string, 'institution': string, 'degree': string, 'year': string}; // Educational background. `id` is a unique identifier for each education record.

2. Define relationships in a key-value pair format:
    Relationship Types:
    - {'personid': string, 'relations

In [102]:
extracted_info = llm(resume_input_text)

print((extracted_info))


 




{
    "entities": [
        {
            "label": "Person",
            "properties": {
                "id": "stanhillrobert",
                "name": "Robert Stanhill (Ramasubramanya M.S.)"
            }
        },
        {
            "label": "Experience",
            "properties": {
                "id": "boschGlobal",
                "role": "Angular Developer",
                "organization": "Bosch Global Software Technologies"
            }
        },
        {
            "label": "Education",
            "properties": {
                "id": "msAi",
                "institution": "RMIT University, Melbourne",
                "degree": "Master in AI",
                "year": "2023-2025"
            }
        },
        {
            "label": "Education",
            "properties": {
                "id": "bscCs",
                "institution": "B.N.M Institute of Technology, Bangalore, India",
                "degree": "Bachelor in Computer Science",
                "

In [103]:
# extracted_info = "your original string with JSON-like content"

# Find the index of the first occurrence of '{'
index_of_brace = extracted_info.find('{')

# Slice the string from the index of '{' if '{' is found
if index_of_brace != -1:
    result_string = '[' + extracted_info[index_of_brace:]
else:
    result_string = extracted_info  # or handle the case when '{' is not found

# Find the index of the last occurrence of '}'
index_of_last_brace = result_string.rfind('}')

# Slice the string up to and including the last '}' and then append a ']'
if index_of_last_brace != -1:
    result_string = result_string[:index_of_last_brace + 1] + ']'
else:
    result_string = result_string  # or handle the case when '}' is not found

# Convert all single quotes ' to double quotes "
result_string = result_string.replace("'", '"')

print(result_string)


[{
    "entities": [
        {
            "label": "Person",
            "properties": {
                "id": "stanhillrobert",
                "name": "Robert Stanhill (Ramasubramanya M.S.)"
            }
        },
        {
            "label": "Experience",
            "properties": {
                "id": "boschGlobal",
                "role": "Angular Developer",
                "organization": "Bosch Global Software Technologies"
            }
        },
        {
            "label": "Education",
            "properties": {
                "id": "msAi",
                "institution": "RMIT University, Melbourne",
                "degree": "Master in AI",
                "year": "2023-2025"
            }
        }]


In [104]:
extracted_info = llm(resume_input_text)

print((extracted_info))


• Winner  of RMIT  2-week  Industry Experience Program  (RMIT – Practera)  


{
    "entities": [
        {
            "label": "Person",
            "properties": {
                "id": "stanhillrobert",
                "name": "Robert Stanhill"
            }
        },
        {
            "label": "Experience",
            "properties": {
                "id": "boschGlobalSoftwareTechnologies",
                "role": "Angular Developer",
                "organization": "Bosch Global Software Technologies"
            }
        },
        {
            "label": "Experience",
            "properties": {
                "id": "crossways",
                "role": "Volunteer",
                "organization": "Crossways"
            }
        },
        {
            "label": "Experience",
            "properties": {
                "id": "rusu",
                "role": "Volunteer",
                "organization": "RUSU"
            }
        },
        {
            "label": "Experi

In [105]:
import json

# Convert the extracted information to a Python dictionary
try:
    parsed_output = json.loads(result_string)
except json.JSONDecodeError:
    print("Error parsing model output as JSON.")
    parsed_output = {"entities": [], "relationships": []}

# Extract entities and relationships
entities = parsed_output["entities"]
relationships = parsed_output["relationships"]

print(parsed_output)


Error parsing model output as JSON.
{'entities': [], 'relationships': []}


In [106]:
import json

json_string = """
[{
    "entities": [
        {
            "label": "Person",
            "properties": {
                "id": "stanhillrobert",
                "name": "Robert Stanhill"
            }
        },
        {
            "label": "Experience",
            "properties": {
                "id": "boschGlobal",
                "role": "Angular Developer",
                "organization": "Bosch Global Software Technologies"
            }
        },
        {
            "label": "Education",
            "properties": {
                "id": "masterInAi",
                "institution": "RMIT University",
                "degree": "Master in AI",
                "year": "2023 -2025"
            }
        },
        {
            "label": "Education",
            "properties": {
                "id": "bachelorInComputerScience",
                "institution": "B.N.M Institute of Technology",
                "degree": "Bachelor in Computer Science",
                "year": "2018 -2022"
            }
        }]
}]
"""

try:
    parsed_data = json.loads(json_string)
    print(type(parsed_data))
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")


<class 'list'>


In [107]:
def create_cypher_queries(entities, relationships):
    queries = []

    # Create queries for entities
    for entity in entities:
        print(entity)
        if entity['label'] == 'Person':
            queries.append(f"CREATE (:Person {{id: '{entity['id']}', name: '{entity['name']}'}})")
        elif entity['label'] == 'Experience':
            queries.append(f"CREATE (:Experience {{id: '{entity['id']}', role: '{entity['role']}', organization: '{entity['organization']}', duration: '{entity['duration']}'}})")
        # Add other entity types here

    # Create queries for relationships
    for relation in relationships:
        head, rel_type, tail = relation.split('|')
        queries.append(f"MATCH (a), (b) WHERE a.id = '{head}' AND b.id = '{tail}' CREATE (a)-[:{rel_type}]->(b)")

    return queries


# Accessing the first item in the list (which is a dictionary)
first_item = parsed_data[0]

# Accessing the 'entities' key in this dictionary
entities = first_item['entities']

# You can then iterate over entities, access specific properties, etc.
for entity in entities:
    print(entity['label'], entity['properties'])


    cypher_queries = create_cypher_queries(entity['properties'],"")
# print(entities)

Person {'id': 'stanhillrobert', 'name': 'Robert Stanhill'}
id


TypeError: string indices must be integers, not 'str'

In [ ]:
from neo4j import GraphDatabase

url = "neo4j+s://42827bf0.databases.neo4j.io"  # Replace with your Neo4j instance URI
username = "neo4j"  # Replace with your username
password = "wwwD7iG-ZWW1YPDBLb_IoxpQxbdgmLSNKztHArPXldY"  # Replace with your password

driver = GraphDatabase.driver(url, auth=(username, password))

def add_data_to_neo4j(queries):
    with driver.session() as session:
        for query in queries:
            session.run(query)

add_data_to_neo4j(cypher_queries)


In [ ]:
from neo4j import GraphDatabase

url = "neo4j+s://04b59de7.databases.neo4j.io"  # Replace with your Neo4j instance URI
username = "neo4j"  # Replace with your username
password = "9aCyMIcwMivgow5OvvkHbJ9svjUkzviHu5du0c5FZbw"  # Replace with your password

driver = GraphDatabase.driver(url, auth=(username, password))

def add_data_to_neo4j(queries):
    with driver.session() as session:
        for query in queries:
            session.run(query)

add_data_to_neo4j(cypher_queries)
